In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats

# Always make it pretty.
plt.style.use('ggplot')
# Requests sends and recieves HTTP requests.
import requests

# Beautiful Soup parses HTML documents in python.
from bs4 import BeautifulSoup
import datetime

In [2]:
file = pd.read_csv('data/Collisions.csv')

In [3]:
file[file['SPEEDING'] == 'Y'].shape[0]
#Number of Accidents with speeding involved

9492

In [4]:
file[(file['SPEEDING'] == 'Y') & (file['FATALITIES'] > 0)].shape[0]

85

In [5]:
file[file['FATALITIES'] > 0].shape[0]

328

In [6]:
print(file['INCDTTM'].isnull().sum())
file['UNDERINFL'].isnull().sum()


0


25721

In [7]:
file[(file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1' )].shape[0]

9315

In [8]:
file[file['SERIOUSINJURIES'] > 0].shape[0]
file['UNDERINFL'].unique()
drunkdracc = file[((file['UNDERINFL'] == 'Y') | (file['UNDERINFL'] == '1'))].shape[0]

In [9]:
infl = file[((file['UNDERINFL'] == 'Y') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['FATALITIES'] > 0))].shape[0]


underinflratio = infl / file[file['FATALITIES'] > 0 ].shape[0]

noninfl = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]

noninfluratio = noninfl / file[file['FATALITIES'] > 0].shape[0]
#print(noninfluratio)
#print(underinflratio)
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Block')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['ADDRTYPE'] == 'Intersection')].shape[0])
#print(file[(file['FATALITIES'] > 0) & (file['LIGHTCOND'] == 'Daylight')].shape[0])
print('drunk driving deaths / drunk driving accidents: ',(infl/drunkdracc))
nondrunkdrivingacccsdeath = file[((file['UNDERINFL'] == 'N') & (file['FATALITIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['FATALITIES'] > 0))].shape[0]
nondrunkdrivingacc = file[(file['UNDERINFL'] == 'N') | (file['UNDERINFL'] == '0')].shape[0]
print(nondrunkdrivingacc)
print('non drunk driving deaths / non drunk driving accidents: ',(nondrunkdrivingacccsdeath/nondrunkdrivingacc))
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc
shared_sample_freq = (nondrunkdrivingacccsdeath + infl) / denominator
shared_sample_variance = numerator * (shared_sample_freq * (1 - shared_sample_freq)) / denominator
difference_in_proportions = stats.norm(0, np.sqrt(shared_sample_variance))
difference_in_sample_proportions = (nondrunkdrivingacccsdeath/nondrunkdrivingacc)/(infl/drunkdracc)

p_value = 1 - difference_in_proportions.cdf(difference_in_sample_proportions)
print(p_value)

drunk driving deaths / drunk driving accidents:  0.00966183574879227
177568
non drunk driving deaths / non drunk driving accidents:  0.001267120201838169
0.0


In [10]:
seriousinjury = file[((file['UNDERINFL'] == 'Y') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['SERIOUSINJURIES'] > 0))].shape[0]
totalseriousinjury = file[file['SERIOUSINJURIES'] > 0].shape[0]
seriousinjurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['SERIOUSINJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['SERIOUSINJURIES'] > 0))].shape[0]
print(seriousinjury/totalseriousinjury)
print('serious injury driving accidents invoving drunk / drunk driving accidents: ',seriousinjury/drunkdracc)
print('serious injury driving accidents not involving drunk/ non drunk driving accidents: ',seriousinjurynondrunk/nondrunkdrivingacc)

0.14145516074450085
serious injury driving accidents invoving drunk / drunk driving accidents:  0.04487385936661299
serious injury driving accidents not involving drunk/ non drunk driving accidents:  0.014276220940710039


In [11]:
injury = file[((file['UNDERINFL'] == 'Y') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '1') & (file['INJURIES'] > 0))].shape[0]
injurynondrunk = file[((file['UNDERINFL'] == 'N') & (file['INJURIES'] > 0) )| ((file['UNDERINFL'] == '0') & (file['INJURIES'] > 0))].shape[0]
totalinjury = file[file['INJURIES'] > 0].shape[0]
print(injury/totalinjury)
print(injury/drunkdracc)
print(injurynondrunk/nondrunkdrivingacc)

0.06567435901758172
0.41825013419216317
0.3051563344746801


In [12]:
print('car accidents involving drunk driver that injury someone',injury)
print('car accidents involving non drunk driver that injury someone',injurynondrunk)
print('car accidents involving drunk driver',drunkdracc)
print('car accidents involving sober driver',nondrunkdrivingacc)
denominator = drunkdracc * nondrunkdrivingacc 
numerator = drunkdracc + nondrunkdrivingacc

car accidents involving drunk driver that injury someone 3896
car accidents involving non drunk driver that injury someone 54186
car accidents involving drunk driver 9315
car accidents involving sober driver 177568


In [13]:

#file['ST_COLCODE'] = pd.to_numeric(file[file.ST_COLCODE.notnull()])
file['ST_COLCODE'] = pd.to_numeric(file['ST_COLCODE'],errors = 'coerce')
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['FATALITIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['SERIOUSINJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 0) & (file['ST_COLCODE'] <= 5)].groupby('ST_COLCODE').count()['X'].sum())
print(file[(file['INJURIES'] > 0) & (file['ST_COLCODE'] >= 10)].groupby('ST_COLCODE').count()['X'].sum())

115
191
823
2050
6673
50121


In [15]:
latlong= file.iloc[0:101,0:2]
latlong['state'] = 'WA'
latlong
#trying to get all the zipcodes of data

,X,Y,state
0,-122.270834,47.539355,WA
1,-122.306332,47.606167,WA
2,-122.269611,47.516849,WA
3,-122.307505,47.612898,WA
4,-122.312430,47.708528,WA
5,-122.328079,47.604161,WA
6,-122.368598,47.670894,WA
7,-122.344507,47.687665,WA
8,-122.334266,47.733225,WA
9,-122.380504,47.672365,WA


In [17]:
file2 = file.copy()
file2['INCDTTM'].str.contains('AM|PM',regex=True)
test2 =file2['INCDTTM']
test3 =test2[file2['INCDTTM'].str.contains('AM|PM',regex=True)]

In [21]:
for idx, my_date in enumerate(test3):
    test3.iloc[idx] = datetime.datetime.strptime(my_date,'%m/%d/%Y %I:%M:%S %p')

In [30]:
test3.max()

datetime.datetime(2019, 7, 29, 12, 26)

In [31]:
test3.min()

datetime.datetime(2004, 1, 10, 1, 27)

In [32]:
url = 'https://api.darksky.net/forecast/'

In [47]:
with open('data/dark_sky.txt', 'r') as file:
    data = file.read().replace('\n', '')
key = 

In [74]:
latlong = '/47.6062,-122.3321,'
##[YYYY]-[MM]-[DD]T[HH]:[MM]:[SS]
time = '[2014]-[01]-[10]T[1]:[27]:[00]'
time2 = '1389317220'
exclude = '?exclude=flags'

In [75]:
finalurl= url+key+latlong+time2
finalurl

'https://api.darksky.net/forecast/c9a626cef1e5d878b0588d229b7934c2/47.6062,-122.3321,1389317220'

In [76]:
r = requests.get(finalurl)

In [88]:
r.status_code

200

In [78]:
r.content

b'{"latitude":47.6062,"longitude":-122.3321,"timezone":"America/Los_Angeles","currently":{"time":1389317220,"summary":"Drizzle","icon":"rain","precipIntensity":0.0136,"precipProbability":0.9,"precipType":"rain","temperature":43.46,"apparentTemperature":41.09,"dewPoint":41.49,"humidity":0.93,"pressure":1007.56,"windSpeed":4.27,"windGust":8.67,"windBearing":150,"cloudCover":1,"uvIndex":0,"visibility":3.572},"hourly":{"summary":"Drizzle in the afternoon and evening.","icon":"rain","data":[{"time":1389254400,"summary":"Partly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":44.98,"apparentTemperature":43.59,"dewPoint":41.41,"humidity":0.87,"pressure":1010.58,"windSpeed":3.41,"windGust":8.11,"windBearing":209,"cloudCover":0.47,"uvIndex":0,"visibility":10},{"time":1389258000,"summary":"Partly Cloudy","icon":"partly-cloudy-night","precipIntensity":0,"precipProbability":0,"temperature":44.31,"apparentTemperature":43.15,"dewPoint":40.29,"humidity":0.8